# House Price Hybrid Boosting Model

In [ ]:
from house_price_utils import *

setup_notebook()

In [ ]:
data, data_test = load_data()
data = data[sorted(data)]
X_raw = data.drop(columns=["SalePrice"])
y = data.SalePrice

In [ ]:
full_data = pd.concat([X_raw, data_test])
features = group_features(full_data)
full_clean_data = clean(full_data, features)
X = full_clean_data[lambda x: x.index.isin(X_raw.index)]
X_test = full_clean_data[lambda x: x.index.isin(data_test.index)]

# Model Experiments


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_error

# 1. Absolute Error Loss

In [ ]:
model = RandomForestRegressor(random_state=0, criterion="absolute_error")

In [ ]:
scores = cross_validate(
    model,
    X,
    y,
    cv=10,
    scoring="neg_mean_absolute_error",
    return_train_score=True,
    n_jobs=3,
)

In [ ]:
-scores["train_score"].mean(), scores["train_score"].std()

In [ ]:
-scores["test_score"].mean(), scores["test_score"].std()

# Observations
* Still overfitting.
* MAE is way slower to fit.

# 2. Grid Search
Let's see if some parameter tuning can help.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=0, shuffle=True
)

In [ ]:
model = RandomForestRegressor(random_state=0)

In [ ]:
model.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "min_samples_leaf": (1, 2, 4, 8, 16),
    "max_depth": (None, 5, 10, 20, 30),
    "max_features": ("log2", "sqrt"),
    "n_estimators": (10, 50, 100, 200),
}
model_grid_search = GridSearchCV(
    model, param_grid=param_grid, n_jobs=4, cv=2, scoring="neg_mean_absolute_error"
)
model_grid_search.fit(X_train, y_train)

In [ ]:
model_grid_search.best_params_

In [ ]:
model_grid_search.best_score_

In [ ]:
mean_absolute_error(y_val, model_grid_search.predict(X_val))

In [ ]:
model.set_params(**model_grid_search.best_params_)
model.set_params(criterion="absolute_error")

## Observations
* No improvement in MAE.

# Diagnostics

In [ ]:
from sklearn.compose import TransformedTargetRegressor

In [ ]:
log_model = TransformedTargetRegressor(model, func=np.log, inverse_func=np.exp)

In [ ]:
log_model.fit(X_train, y_train)

In [ ]:
yhat = log_model.predict(X_val)
mean_absolute_error(y_val, yhat)

In [ ]:
fig = residual_plots(y_val, yhat)

# 3. Additive Residual Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
class TwoStageModel:
    def __init__(self, model, resid_model):
        self.model = model
        self.resid_model = resid_model

    def fit(self, X, y):
        self.model.fit(X, y)
        yhat = self.model.predict(X)
        X2 = np.column_stack([yhat, yhat**2])
        self.resid_model.fit(X2, y - yhat)

    def predict(self, X):
        yhat = self.model.predict(X)
        X2 = np.column_stack([yhat, yhat**2])
        return yhat + self.resid_model.predict(X2)

In [ ]:
resid_model = LinearRegression()

In [ ]:
two_stage_model = TwoStageModel(log_model, resid_model)

In [ ]:
two_stage_model.fit(X_train, y_train)
yhat = two_stage_model.predict(X_train)

In [ ]:
mean_absolute_error(y_train, yhat)

In [ ]:
fig = residual_plots(y_train, yhat)

In [ ]:
yhat = two_stage_model.predict(X_val)

In [ ]:
mean_absolute_error(y_val, yhat)

In [ ]:
fig = residual_plots(y_val, yhat)

In [ ]:
two_stage_model.fit(X, y)
yhat = two_stage_model.predict(X)

In [ ]:
mean_absolute_error(y, yhat)

In [ ]:
fig = residual_plots(y, yhat)

In [ ]:
resid_df = pd.DataFrame({"resid": y - yhat, "y": y, "yhat": yhat}).join(X_raw)

In [ ]:
resid_df.sort_values("resid")

In [ ]:
X_raw.SaleCondition.value_counts()

In [ ]:
data_test.SaleCondition.value_counts()

# Submit

In [ ]:
if False:
    import kaggle

    create_submission(two_stage_model, X_test)
    kaggle.api.competition_submit(
        "submission.csv", "Two stage model", "home-data-for-ml-course"
    )